<a href="https://colab.research.google.com/github/alexdoss/git_practivce/blob/main/xray_weekday_config.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import datetime
df=pd.read_csv('/content/df_all_weekdays.csv', delimiter=',')
print(df.shape)
df['pos_edition']=df['point_of_sales'].astype(str) + '_' + df['edition'].astype(str)
df.shape

(3023204, 12)


(3023204, 13)

In [4]:
#récuperation provider files
df_providers = pd.read_parquet('/content/providers (4).parquet')
df_providers.columns.values.tolist()

['ALLOW_FORECAST_FRIDAY',
 'ALLOW_FORECAST_MONDAY',
 'ALLOW_FORECAST_SATURDAY',
 'ALLOW_FORECAST_SUNDAY',
 'ALLOW_FORECAST_THURSDAY',
 'ALLOW_FORECAST_TUESDAY',
 'ALLOW_FORECAST_WEDNESDAY',
 'Adresse',
 'Code_INSEE',
 'Code_Kiosque',
 'Code_département',
 'Code_postal',
 'Code_région',
 'DEBEXCOPEN',
 'DEBSUSP',
 'DEBSUSP2',
 'Date_fermeture',
 'Date_ouverture',
 'Département',
 'Dépositaire',
 'Edition',
 'Edition_Code',
 'Edition_Name',
 'Edition_defaut',
 'FINEXCOPEN',
 'FINSUSP',
 'FINSUSP2',
 'Implantation',
 'Kiosque',
 'Latitude',
 'Localité',
 'Longitude',
 'Nom_diffuseur',
 'Ouvert_dimanche',
 'Ouvert_ferie',
 'Ouvert_jeudi',
 'Ouvert_lundi',
 'Ouvert_mardi',
 'Ouvert_mercredi',
 'Ouvert_samedi',
 'Ouvert_vendredi',
 'Perimetre',
 'Région',
 'SERVICE_FRIDAY',
 'SERVICE_MONDAY',
 'SERVICE_SATURDAY',
 'SERVICE_SUNDAY',
 'SERVICE_THURSDAY',
 'SERVICE_TUESDAY',
 'SERVICE_WEDNESDAY',
 'Secteur',
 'Segment',
 'deposi_nim']

In [5]:
df_providers_short=df_providers[['Kiosque','Région']].drop_duplicates()
df_providers_short.value_counts()

,,count
Kiosque,Région,
21719,AUVERGNE-RHONE-ALPES,1
25143,AUVERGNE-RHONE-ALPES,1
25145,AUVERGNE-RHONE-ALPES,1
25147,AUVERGNE-RHONE-ALPES,1
25148,AUVERGNE-RHONE-ALPES,1
...,...,...
23704,BOURGOGNE-FRANCHE-COMTE,1
23707,AUVERGNE-RHONE-ALPES,1
23708,AUVERGNE-RHONE-ALPES,1


In [6]:

df_providers_short = df_providers_short.rename(columns={'Kiosque': 'point_of_sales'})
df_providers_short.head()

,point_of_sales,Région
0,9669,BOURGOGNE-FRANCHE-COMTE
1,23764,BOURGOGNE-FRANCHE-COMTE
2,23732,BOURGOGNE-FRANCHE-COMTE
3,23731,AUVERGNE-RHONE-ALPES
4,23729,AUVERGNE-RHONE-ALPES


In [7]:
df_providers_short['point_of_sales']= df_providers_short['point_of_sales'].astype('Int64')
df_providers_short.head()

,point_of_sales,Région
0,9669,BOURGOGNE-FRANCHE-COMTE
1,23764,BOURGOGNE-FRANCHE-COMTE
2,23732,BOURGOGNE-FRANCHE-COMTE
3,23731,AUVERGNE-RHONE-ALPES
4,23729,AUVERGNE-RHONE-ALPES


In [8]:
#merge sales and providers (focus on "region")
df_merge=pd.merge(df, df_providers_short, on='point_of_sales', how ='left')
df_merge.head()

,Unnamed: 0,pk,point_of_sales,edition,parution,date,sales,provided_real,unsold_real,soldout_real,date_time,week_day,pos_edition,Région
0,5030,8867-244-417810-80,8867,244,417810,2021-11-13,2.0,3,1.0,False,2021-11-13,Saturday,8867_244,BOURGOGNE-FRANCHE-COMTE
1,5031,8867-254-421450-80,8867,254,421450,2021-11-13,1.0,2,1.0,False,2021-11-13,Saturday,8867_254,BOURGOGNE-FRANCHE-COMTE
2,5032,8867-260-423270-80,8867,260,423270,2021-11-13,3.0,4,1.0,False,2021-11-13,Saturday,8867_260,BOURGOGNE-FRANCHE-COMTE
3,5033,8867-244-417811-80,8867,244,417811,2021-11-14,2.0,4,2.0,False,2021-11-14,Sunday,8867_244,BOURGOGNE-FRANCHE-COMTE
4,5034,8867-254-421451-80,8867,254,421451,2021-11-14,1.0,2,1.0,False,2021-11-14,Sunday,8867_254,BOURGOGNE-FRANCHE-COMTE


In [11]:
#FLORENTINA SUM OF SALES PER REGION
df_parution_region=df_merge[['Région','sales', 'provided_real', 'unsold_real']].groupby(['Région']).sum()
df_parution_region.head()

,sales,provided_real,unsold_real
Région,,,
AUVERGNE-RHONE-ALPES,49106237.0,63539153,14432916.0
BOURGOGNE-FRANCHE-COMTE,6046443.0,7773608,1727165.0
OCCITANIE,10255.0,15406,5151.0


In [28]:
#Number of parutions per region
parutions_per_region=df_merge[['Région', 'date']].groupby(['Région']).nunique('date')
parutions_per_region.head()

,date
Région,
AUVERGNE-RHONE-ALPES,1084
BOURGOGNE-FRANCHE-COMTE,1083
OCCITANIE,1076


In [25]:
#check occitanie
df_merge_BOURGOGNE_FRANCHE_COMTE= df_merge[df_merge['Région']=='BOURGOGNE-FRANCHE-COMTE']
df_merge_BOURGOGNE_FRANCHE_COMTE.head()
df_merge_BOURGOGNE_FRANCHE_COMTE.to_csv('df_merge_BOURGOGNE_FRANCHE_COMTE.csv', index=True)

In [27]:
#check auvergne rhone alpes
df_merge_AUVERGNE_RHONE_ALPES= df_merge[df_merge['Région']=='AUVERGNE-RHONE-ALPES']
df_merge_AUVERGNE_RHONE_ALPES.head()
df_merge_BOURGOGNE_FRANCHE_COMTE.to_csv('df_merge_AUVERGNE_RHONE_ALPES.csv', index=True)

In [ ]:
#FRONTIERE

In [ ]:
df_date_weekday=df[['date','week_day']].drop_duplicates()
df_date_weekday.head()

,date,week_day
0,2021-11-13,Saturday
3,2021-11-14,Sunday
6,2021-11-15,Monday
9,2021-11-16,Tuesday
12,2021-11-17,Wednesday


In [ ]:
#nbr of issues per weekday
nbIssues_unique_Pdv_per_weekday=df[['week_day','date']].groupby(['week_day']).nunique('date')
print(nbIssues_unique_Pdv_per_weekday)

           date
week_day       
Friday      155
Monday      155
Saturday    156
Sunday      155
Thursday    155
Tuesday     155
Wednesday   154


In [ ]:
#unique PoS and PoS_edition per issues (will be used to have the sum of unique PoS / PoS_edition per issues)
df_parution_pos_focus=df[['date', 'point_of_sales', 'pos_edition']].groupby(['date']).nunique()
df_parution_pos_focus_weekday=pd.merge(df_parution_pos_focus, df_date_weekday, on='date', how ='outer')
df_parution_pos_focus_weekday.to_csv('df_parution_pos_focus_weekday.csv', index=True)
df_parution_pos_focus_weekday.head()

,date,point_of_sales,pos_edition,week_day
0,2021-11-13,2342,3113,Saturday
1,2021-11-14,2066,2741,Sunday
2,2021-11-15,2152,2846,Monday
3,2021-11-16,2466,3244,Tuesday
4,2021-11-17,2377,3122,Wednesday


In [ ]:
#FLORENTINA sum of unique PoS and PoS_edition per weekday
df_parution_pos_focus_weekday.groupby(['week_day']).sum()

,date,point_of_sales,pos_edition
week_day,,,
Friday,2021-11-192021-11-262021-12-032021-12-102021-1...,353180,459238
Monday,2021-11-152021-11-222021-11-292021-12-062021-1...,304366,397438
Saturday,2021-11-132021-11-202021-11-272021-12-042021-1...,340077,445322
Sunday,2021-11-142021-11-212021-11-282021-12-052021-1...,298960,393493
Thursday,2021-11-182021-11-252021-12-022021-12-092021-1...,344152,446953
Tuesday,2021-11-162021-11-232021-11-302021-12-072021-1...,343816,447530
Wednesday,2021-11-172021-11-242021-12-012021-12-082021-1...,332817,433230


In [ ]:
#FLORENTINA mean of unique PoS and PoS_edition per weekday
df_parution_pos_focus_weekday[['week_day','point_of_sales','pos_edition']].groupby(['week_day']).mean()

,point_of_sales,pos_edition
week_day,,
Friday,2278.580645,2962.825806
Monday,1963.651613,2564.116129
Saturday,2179.980769,2854.628205
Sunday,1928.774194,2538.664516
Thursday,2220.335484,2883.567742
Tuesday,2218.167742,2887.290323
Wednesday,2161.149351,2813.181818


In [ ]:
df_parution_sales_focus=df[['date', 'sales', 'provided_real', 'unsold_real','soldout_real']].groupby(['date']).sum()
df_parution_sales_focus_weekday=pd.merge(df_parution_sales_focus, df_date_weekday, on='date', how ='outer')
df_parution_sales_focus_weekday.to_csv('df_parution_sales_focus_weekday.csv', index=True)
df_parution_sales_focus_weekday.head()

,date,sales,provided_real,unsold_real,soldout_real,week_day
0,2021-11-13,51850.0,66947,15097.0,256,Saturday
1,2021-11-14,117389.0,139469,22080.0,229,Sunday
2,2021-11-15,49223.0,63167,13944.0,227,Monday
3,2021-11-16,50712.0,65250,14538.0,263,Tuesday
4,2021-11-17,49317.0,64282,14965.0,240,Wednesday


In [ ]:
#FLORENTINA sum of "sales" kpi per weekday
df_parution_sales_focus_weekday[['week_day','sales', 'provided_real', 'unsold_real', 'soldout_real']].groupby(['week_day']).sum()

,sales,provided_real,unsold_real,soldout_real
week_day,,,,
Friday,6806782.0,9014454,2207672.0,40413
Monday,6544435.0,8753874,2209439.0,34338
Saturday,7055597.0,9277546,2221949.0,40313
Sunday,15560501.0,19017737,3457236.0,32918
Thursday,6509592.0,8605095,2095503.0,39979
Tuesday,6499274.0,8570846,2071572.0,41777
Wednesday,6396378.0,8391983,1995605.0,39863


In [ ]:
#FLORENTINA mean of "sales" kpi per weekday
df_parution_sales_focus_weekday[['week_day','sales', 'provided_real', 'unsold_real', 'soldout_real']].groupby(['week_day']).mean()

,sales,provided_real,unsold_real,soldout_real
week_day,,,,
Friday,43914.722581,58157.767742,14243.045161,260.729032
Monday,42222.161290,56476.606452,14254.445161,221.535484
Saturday,45228.185897,59471.448718,14243.262821,258.416667
Sunday,100390.329032,122695.077419,22304.748387,212.374194
Thursday,41997.367742,55516.741935,13519.374194,257.929032
Tuesday,41930.800000,55295.780645,13364.980645,269.529032
Wednesday,41534.922078,54493.396104,12958.474026,258.850649
